训练并导出RNN模型

In [1]:
import pandas as pd
import mylib
import torch

df = pd.read_excel("../datasets/flowspeeds.xlsx")

In [2]:
device = mylib.choose_device()
print(device)

mps


## 定义模型

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.rnn = nn.LSTM(input_size=28, hidden_size=64, batch_first=True)
        self.batchnorm = nn.BatchNorm1d(64)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, input):
        # Shape of input is (batch_size,1, 28, 28)
        # converting shape of input to (batch_size, 28, 28)
        # as required by RNN when batch_first is set True
        input = input.reshape(-1, 28, 28)
        output, hidden = self.rnn(input)

        # RNN output shape is (seq_len, batch, input_size)
        # Get last output of RNN
        output = output[:, -1, :]
        output = self.batchnorm(output)
        output = self.dropout1(output)
        output = self.fc1(output)
        output = F.relu(output)
        output = self.dropout2(output)
        output = self.fc2(output)
        output = F.log_softmax(output, dim=1)
        return output


# 加载数据
t = df['TIME']
x = df["154_FT_101.PV"]
y = df["154_FT_101.PV"].shift(1)



In [16]:

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [17]:
from torch.optim.lr_scheduler import StepLR
import argparse
from torchvision import datasets, transforms

torch.manual_seed(1)

device = torch.device("mps")

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../datasets', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../datasets', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=1000, shuffle=True)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=0.1)

scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1, 14 + 1):
    train(model=model, device=device, train_loader=train_loader, optimizer=optimizer, epoch=epoch)
    test(model=model, device=device, test_loader=test_loader)
    scheduler.step()

torch.save(model.state_dict(), "./output/model/mnist_rnn.pt")


/Users/mozhou/.pyenv/versions/3.11.3/lib/python3.11/site-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Train Epoch: 1 [64/60000 (0%)]	Loss: 2.449428
Train Epoch: 1 [128/60000 (0%)]	Loss: 2.373624
Train Epoch: 1 [192/60000 (0%)]	Loss: 2.320312
Train Epoch: 1 [256/60000 (0%)]	Loss: 2.341782
Train Epoch: 1 [320/60000 (1%)]	Loss: 2.382721
Train Epoch: 1 [384/60000 (1%)]	Loss: 2.394810
Train Epoch: 1 [448/60000 (1%)]	Loss: 2.320387
Train Epoch: 1 [512/60000 (1%)]	Loss: 2.307117
Train Epoch: 1 [576/60000 (1%)]	Loss: 2.384568
Train Epoch: 1 [704/60000 (1%)]	Loss: 2.280662
Train Epoch: 1 [768/60000 (1%)]	Loss: 2.311028
Train Epoch: 1 [832/60000 (1%)]	Loss: 2.326630
Train Epoch: 1 [896/60000 (1%)]	Loss: 2.269225
Train Epoch: 1 [960/60000 (2%)]	Loss: 2.307491
Train Epoch: 1 [1024/60000 (2%)]	Loss: 2.249155
Train Epoch: 1 [1088/60000 (2%)]	Loss: 2.230831
Train Epoch: 1 [1152/60000 (2%)]	Loss: 2.273053
Train Epoch: 1 [1216/60000 (2%)]	Loss: 2.340099
Train Epoch: 1 [1344/60000 (2%)]	Loss: 2.322458
Train Epoch: 1 [1408/60000 (2%)]	Loss: 2.250056
Train Epoch: 1 [1472/60000 (2%)]	Loss: 2.245269
Train E